# **Instituto de Informática - UFG**

Disciplina: Inteligência Artificial

Prof. Cedric Luiz de Carvalho

Grupo ( A )

---


Aluna(o): Luana Guedes Barros Martins

---

Aluna(o): Ruan Chaves Rodrigues

---

Aluna(o): Thiago Monteles de Souza

---

# Atividade Prática 06 - Processamento de Linguagem Natural

Particionamento das atividades:
- Luana Guedes Barros Martins: Desenvolvedor
- Ruan Chaves Rodrigues: Analista de dados
- Thiago Monteles de Souza: Gerente do Projeto

In [1]:
!pip install gensim

In [14]:
# Imports

from collections import defaultdict
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess
import numpy as np

## Descrição do Problema

Utilizando as técnicas estudadas, será desenvolvido uma ferramenta que permita a comparação de pequenos textos. Devem ser definidas métricas de comparação que classifique os textos segundo critérios de similaridade.

A ferramenta deverá ser aplicada aos Estudos Dirigidos desenvolvidos ao longo do curso de Inteligência Artificial 2019-3. Para cada estudo, deve-se construir uma matriz que mostre a similaridade entre os documentos. Depois, definir um documento de referência e traçar um gráfico de similaridade dos demais em relação a ele.

## Tratamento dos Dados

In [15]:
docs = [ "Feita a comparação com igual período do ano passado, registaram-se menos 29 acidentes e menos dois mortos.",
       "Em comparação com o ano anterior, registaram-se menos 29 acidentes e menos duas vítimas mortais.",
       "Presidente da República fez uma série de avisos ao novo primeiro-ministro na tomada de posse do XXI Governo Constitucional.",
       "Foi naquele local que António Costa tomou posse do XXI Governo Constitucional."
       ]


In [16]:
mydict = corpora.Dictionary([simple_preprocess(line) for line in docs])

In [18]:
corpus = [mydict.doc2bow(simple_preprocess(line)) for line in docs]

In [19]:
for doc in corpus:
    print([[mydict[id], freq] for id, freq in doc])

[['acidentes', 1], ['ano', 1], ['com', 1], ['comparação', 1], ['do', 1], ['dois', 1], ['feita', 1], ['igual', 1], ['menos', 2], ['mortos', 1], ['passado', 1], ['período', 1], ['registaram', 1], ['se', 1]]
[['acidentes', 1], ['ano', 1], ['com', 1], ['comparação', 1], ['menos', 2], ['registaram', 1], ['se', 1], ['anterior', 1], ['duas', 1], ['em', 1], ['mortais', 1], ['vítimas', 1]]
[['do', 1], ['ao', 1], ['avisos', 1], ['constitucional', 1], ['da', 1], ['de', 2], ['fez', 1], ['governo', 1], ['ministro', 1], ['na', 1], ['novo', 1], ['posse', 1], ['presidente', 1], ['primeiro', 1], ['república', 1], ['série', 1], ['tomada', 1], ['uma', 1], ['xxi', 1]]
[['do', 1], ['constitucional', 1], ['governo', 1], ['posse', 1], ['xxi', 1], ['antónio', 1], ['costa', 1], ['foi', 1], ['local', 1], ['naquele', 1], ['que', 1], ['tomou', 1]]


## Definição do Modelo

In [20]:
tfidf = models.TfidfModel(corpus, smartirs='ntc')

In [21]:
for doc in tfidf[corpus]:
    print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])

[['acidentes', 0.19], ['ano', 0.19], ['com', 0.19], ['comparação', 0.19], ['do', 0.1], ['dois', 0.33], ['feita', 0.33], ['igual', 0.33], ['menos', 0.37], ['mortos', 0.33], ['passado', 0.33], ['período', 0.33], ['registaram', 0.19], ['se', 0.19]]
[['acidentes', 0.2], ['ano', 0.2], ['com', 0.2], ['comparação', 0.2], ['menos', 0.4], ['registaram', 0.2], ['se', 0.2], ['anterior', 0.35], ['duas', 0.35], ['em', 0.35], ['mortais', 0.35], ['vítimas', 0.35]]
[['do', 0.07], ['ao', 0.23], ['avisos', 0.23], ['constitucional', 0.13], ['da', 0.23], ['de', 0.47], ['fez', 0.23], ['governo', 0.13], ['ministro', 0.23], ['na', 0.23], ['novo', 0.23], ['posse', 0.13], ['presidente', 0.23], ['primeiro', 0.23], ['república', 0.23], ['série', 0.23], ['tomada', 0.23], ['uma', 0.23], ['xxi', 0.13]]
[['do', 0.11], ['constitucional', 0.2], ['governo', 0.2], ['posse', 0.2], ['xxi', 0.2], ['antónio', 0.35], ['costa', 0.35], ['foi', 0.35], ['local', 0.35], ['naquele', 0.35], ['que', 0.35], ['tomou', 0.35]]


In [22]:
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=12)